In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset Preprocessing

In [2]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 35.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 80.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-st

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="your_api_key_here")
project = rf.workspace("mushroom-species").project("defects-in-timber")
version = project.version(5)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to defects-in-timber-5 in yolov8:: 100%|██████████| 11946/11946 [00:01<00:00, 8650.70it/s] 


In [6]:
import shutil
if os.path.exists("defects-in-timber-5"):
    shutil.rmtree("defects-in-timber-5")

In [ ]:
os.environ['KAGGLE_USERNAME'] = "kaggle_username_here"
os.environ['KAGGLE_KEY'] = "kaggle_key_here"

In [4]:
!kaggle datasets download umarfarooq211203/defects-in-timber

Dataset URL: https://www.kaggle.com/datasets/umarfarooq211203/defects-in-timber
License(s): CC0-1.0
 77%|██████████████████████████████▊         | 338M/438M [00:00<00:00, 1.18GB/s]
100%|████████████████████████████████████████| 438M/438M [00:00<00:00, 1.18GB/s]


In [6]:
import zipfile
import os

# Path to your zip file
zip_path = '/kaggle/working/defects-in-timber.zip'

# Directory where you want to extract the files
extract_dir = '/kaggle/working/defects-in-timber-5'

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


Files extracted to /kaggle/working/defects-in-timber-5


# Data Evaluation

In [8]:
import os

DATASET_PATH = "/kaggle/working/defects-in-timber-5"
splits = ["train", "valid", "test"]

NUM_CLASSES = 4

def count_images_per_class(split):
    label_dir = os.path.join(DATASET_PATH, split, "labels")

    img_count = {i: 0 for i in range(NUM_CLASSES)}

    for lbl in os.listdir(label_dir):
        if not lbl.endswith(".txt"):
            continue
        
        classes_in_file = set()
        with open(os.path.join(label_dir, lbl)) as f:
            for line in f:
                parts = line.strip().split()
                try:
                    cls = int(float(parts[0]))    # SAFE PARSE
                    classes_in_file.add(cls)
                except:
                    continue

        for cls in classes_in_file:
            img_count[cls] += 1

    return img_count


def count_objects_per_class(split):
    label_dir = os.path.join(DATASET_PATH, split, "labels")

    obj_count = {i: 0 for i in range(NUM_CLASSES)}

    for lbl in os.listdir(label_dir):
        if not lbl.endswith(".txt"):
            continue
        with open(os.path.join(label_dir, lbl)) as f:
            for line in f:
                parts = line.strip().split()
                try:
                    cls = int(float(parts[0]))
                    obj_count[cls] += 1
                except:
                    continue

    return obj_count


# ---- PRINT COUNTS ----
for split in splits:
    print(f"\n==== {split.upper()} ====")
    print("Images per class:", count_images_per_class(split))
    print("Objects per class:", count_objects_per_class(split))



==== TRAIN ====
Images per class: {0: 2628, 1: 2772, 2: 3431, 3: 2866}
Objects per class: {0: 3246, 1: 3897, 2: 5431, 3: 3445}

==== VALID ====
Images per class: {0: 365, 1: 376, 2: 488, 3: 406}
Objects per class: {0: 454, 1: 510, 2: 778, 3: 494}

==== TEST ====
Images per class: {0: 254, 1: 263, 2: 305, 3: 269}
Objects per class: {0: 307, 1: 357, 2: 506, 3: 332}


# Augmentation

In [5]:
import os
import cv2
import random
from albumentations import (
    HorizontalFlip, VerticalFlip, Rotate, ShiftScaleRotate, Compose
)

DATASET_PATH = "/kaggle/working/defects-in-timber-5"
splits = ["train","test","valid"]

TARGET_COUNT = {"train":3200,"test":300,"valid":450}
CLASSES_TO_AUGMENT = [0, 3]   
#CLASSES_TO_AUGMENT = [1]   
# Crack, knot_with_crack

transform = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    Rotate(limit=20, p=0.5),
    ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.10,
        rotate_limit=0,
        border_mode=cv2.BORDER_CONSTANT,
        p=0.5
    )
], bbox_params={'format': 'yolo', 'label_fields': ['class_labels']})


ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.


In [6]:
def load_annotations(label_file):
    bboxes, class_labels = [], []

    with open(label_file) as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) != 5:
                continue
            
            try:
                cls = int(float(parts[0]))
                x, y, w, h = map(float, parts[1:])
            except:
                continue

            class_labels.append(cls)
            bboxes.append([x, y, w, h])  # already normalized YOLO
    return bboxes, class_labels


def save_annotations(label_file, bboxes, class_labels):
    with open(label_file, "w") as f:
        for cls, (x, y, w, h) in zip(class_labels, bboxes):
            f.write(f"{cls} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")


In [7]:
def count_objects(split):
    return count_objects_per_class(split)

for split in splits:
    print(f"\n=== AUGMENTING: {split.upper()} ===")

    label_dir = os.path.join(DATASET_PATH, split, "labels")
    image_dir = os.path.join(DATASET_PATH, split, "images")

    counts = count_objects(split)

    for cls in CLASSES_TO_AUGMENT:
        need = TARGET_COUNT[split] - counts[cls]
        if need <= 0:
            print(f"Class {cls} already >= target.")
            continue

        print(f"Class {cls}: Need {need} additional objects.")

        # Get all files containing this class
        label_files = []
        for f in os.listdir(label_dir):
            if f.endswith(".txt"):
                txt_path = os.path.join(label_dir, f)
                content = open(txt_path).read().split()
                if str(cls) in content:
                    label_files.append(f)

        if len(label_files) == 0:
            print(f"No images containing class {cls}.")
            continue

        added = 0
        idx = 0

        while added < need:
            lbl = label_files[idx % len(label_files)]
            base = lbl.replace(".txt", "")

            # find image
            img_path = None
            for ext in (".jpg", ".png", ".jpeg"):
                fp = os.path.join(image_dir, base + ext)
                if os.path.exists(fp):
                    img_path = fp
                    extension = ext
                    break
            if img_path is None:
                idx += 1
                continue

            image = cv2.imread(img_path)

            bboxes, class_labels = load_annotations(os.path.join(label_dir, lbl))
            if len(bboxes) == 0:
                idx += 1
                continue

            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)

            if len(augmented["bboxes"]) == 0:
                idx += 1
                continue

            new_img = augmented["image"]
            new_bboxes = augmented["bboxes"]
            new_labels = augmented["class_labels"]

            new_name = f"{base}_aug_{added}"
            cv2.imwrite(os.path.join(image_dir, new_name + extension), new_img)
            save_annotations(os.path.join(label_dir, new_name + ".txt"), new_bboxes, new_labels)

            added += new_labels.count(cls)
            idx += 1

        print(f"✓ Added approximately {added} objects for class {cls}\n")



=== AUGMENTING: TRAIN ===
Class 0: Need 1704 additional objects.
✓ Added approximately 1705 objects for class 0

Class 3: Need 554 additional objects.
✓ Added approximately 554 objects for class 3


=== AUGMENTING: TEST ===
Class 0: Need 167 additional objects.
✓ Added approximately 167 objects for class 0

Class 3: Need 80 additional objects.
✓ Added approximately 80 objects for class 3


=== AUGMENTING: VALID ===
Class 0: Need 254 additional objects.
✓ Added approximately 254 objects for class 0

Class 3: Need 38 additional objects.
✓ Added approximately 38 objects for class 3



In [8]:

# ---- PRINT COUNTS ----
for split in splits:
    print(f"\n==== {split.upper()} ====")
    print("Images per class:", count_images_per_class(split))
    print("Objects per class:", count_objects_per_class(split))


==== TRAIN ====
Images per class: {0: 2628, 1: 2772, 2: 3431, 3: 2866}
Objects per class: {0: 3246, 1: 3897, 2: 5431, 3: 3445}

==== TEST ====
Images per class: {0: 254, 1: 263, 2: 305, 3: 269}
Objects per class: {0: 307, 1: 357, 2: 506, 3: 332}

==== VALID ====
Images per class: {0: 365, 1: 376, 2: 488, 3: 406}
Objects per class: {0: 454, 1: 510, 2: 778, 3: 494}


# Storing Dataset in Kaggle

In [9]:
import shutil

dataset_folder = "/kaggle/working/defects-in-timber-5"
output_zip = "/kaggle/working/defects-in-timber.zip"

shutil.make_archive(base_name=output_zip.replace(".zip",""),
                    format='zip',
                    root_dir=dataset_folder)


'/kaggle/working/defects-in-timber.zip'

In [11]:
        # Example of creating a metadata file
dataset_dir = "/kaggle/dataset/defects-in-timber"
os.makedirs(dataset_dir, exist_ok=True)
import json
metadata = {
    "title": "Defects-in-Timber Dataset",
    "id": f"{os.environ['KAGGLE_USERNAME']}/defects-in-timber",
    "licenses": [{"name": "CC0-1.0"}]  # <-- This is required
}
with open(os.path.join(dataset_dir, "dataset-metadata.json"), "w") as f:
    json.dump(metadata, f, indent=4)

In [12]:
src_folder = "/kaggle/working/defects-in-timber-5"
dst_folder = "/kaggle/dataset/defects-in-timber"

# Copy all files and subfolders
for item in os.listdir(src_folder):
    s = os.path.join(src_folder, item)
    d = os.path.join(dst_folder, item)
    if os.path.isdir(s):
        shutil.copytree(s, d, dirs_exist_ok=True)
    else:
        shutil.copy2(s, d)

print("Dataset files copied!")

Dataset files copied!


In [13]:
!kaggle datasets create -p "{dataset_dir}" --dir-mode zip

Starting upload for file valid.zip
100%|██████████████████████████████████████| 48.2M/48.2M [00:00<00:00, 60.5MB/s]
Upload successful: valid.zip (48MB)
Starting upload for file README.dataset.txt
100%|████████████████████████████████████████████| 155/155 [00:00<00:00, 360B/s]
Upload successful: README.dataset.txt (155B)
Starting upload for file test.zip
100%|██████████████████████████████████████| 34.8M/34.8M [00:00<00:00, 52.7MB/s]
Upload successful: test.zip (35MB)
Starting upload for file README.roboflow.txt
100%|██████████████████████████████████████| 1.17k/1.17k [00:00<00:00, 2.79kB/s]
Upload successful: README.roboflow.txt (1KB)
Starting upload for file data.yaml
100%|████████████████████████████████████████████| 313/313 [00:00<00:00, 689B/s]
Upload successful: data.yaml (313B)
Starting upload for file train.zip
100%|█████████████████████████████████████████| 354M/354M [00:02<00:00, 128MB/s]
Upload successful: train.zip (354MB)
Your private Dataset is being created. Please check 

# Model Architecture Tiny Vit YOLO

## Dependencies

In [12]:
!pip install ultralytics timm -q

## Tiny Vit Backbone Architecture

In [13]:
import torch
import torch.nn as nn
from ultralytics import YOLO
import timm


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [14]:
import torch
import torch.nn as nn
import timm

class TinyViTBackboneYOLO(nn.Module):
    def __init__(self, model_name='tiny_vit_11m_224.in1k', pretrained=True):
        super().__init__()
        # Load TinyViT with multi-scale outputs
        self.model = timm.create_model(model_name, pretrained=pretrained, features_only=True)
        self.out_indices = (-3, -2, -1)  # last 3 feature maps

        # Adapter convs to match YOLOv8n head channels
        feature_info = self.model.feature_info
        self.adapters = nn.ModuleList([
            nn.Conv2d(feature_info[i]["num_chs"], ch, 1)
            for i, ch in enumerate([256, 512, 1024])
        ])

    def forward(self, x):
        features = self.model(x)
        out = [self.adapters[i](features[idx]) for i, idx in enumerate(self.out_indices)]
        return out


## Build Model

In [15]:
from ultralytics import YOLO

# Load pretrained YOLOv8n
yolo_model = YOLO("yolov8n.pt")

# Replace backbone
backbone = TinyViTBackboneYOLO(pretrained=True)
yolo_model.model.backbone = backbone


model.safetensors:   0%|          | 0.00/44.1M [00:00<?, ?B/s]

In [16]:
!pip install torchinfo -q

In [ ]:
from torchinfo import summary

# Assume input image size 640x640 with 3 channels
summary(yolo_model.model, input_size=(1, 3, 640, 640))


In [ ]:
yolo2 = YOLO("yolov8n.pt")
summary(yolo2.model)


## Training

In [ ]:
yolo_model.train(
    data="defects-in-timber-5/data.yaml",
    imgsz=640,
    batch=8,
    epochs=100,
    patience=5,
    optimizer="AdamW",
    device=0
)


In [ ]:
import shutil
folder_path = "/kaggle/working/runs"  
archive_name = "/kaggle/working/yolov8_tinyvit_results"  
shutil.make_archive(archive_name, 'zip', folder_path)


In [ ]:
os.remove(archive_name+".zip")


In [ ]:
metrics = yolo_model.val(data="defects-in-timber-5/data.yaml", batch=2, imgsz=640)
print(metrics)

In [ ]:
# Evaluate on test dataset
test_metrics = yolo_model.val(data="defects-in-timber-5/data.yaml", split="test", batch=2, imgsz=640)
print(test_metrics)


## Training 2

In [ ]:
yolo_model = YOLO("/kaggle/working/runs/detect/train2/weights/best.pt")

In [ ]:
yolo_model.train(
    data="defects-in-timber-5/data.yaml",
    imgsz=640,
    batch=8,
    epochs=20,
    patience=5,
    optimizer="AdamW",
    device=0
)


In [ ]:
yolo_model = YOLO("/kaggle/working/runs/detect/train7/weights/best.pt")

In [ ]:
# Evaluate on test dataset
test_metrics = yolo_model.val(data="defects-in-timber-5/data.yaml", split="test", batch=8, imgsz=640)
print(test_metrics)


In [ ]:
metrics = yolo_model.val(data="defects-in-timber-5/data.yaml", batch=8, imgsz=640)
print(metrics)

## Training 3

In [ ]:
yolo_model = YOLO("/kaggle/working/runs/detect/train7/weights/best.pt")

In [ ]:
yolo_model.train(
    data="defects-in-timber-5/data.yaml",
    imgsz=640,
    batch=8,
    epochs=60,
    patience=5,
    optimizer="AdamW",
    device=0
)


In [ ]:
yolo_model = YOLO("/kaggle/working/runs/detect/train10/weights/best.pt")

In [ ]:
metrics = yolo_model.val(data="defects-in-timber-5/data.yaml", batch=2, imgsz=640)
print(metrics)

In [ ]:
# Evaluate on test dataset
test_metrics = yolo_model.val(data="defects-in-timber-5/data.yaml", split="test", batch=2, imgsz=640)
print(test_metrics)
